In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
class KalmanFilter(object):
    def __init__(self, dt, u_x,u_y, std_acc, x_std_meas, y_std_meas):
        
        #x_std_meas ans y_std_meas are standard diviation about x and y axis

        self.dt = dt

        #Input variables in 2D  
        self.u = np.matrix([[u_x],[u_y]])

        ### Initialisation ###
        self.x = np.matrix([[0], [0], [0], [0]])

        self.F = np.matrix([[1, 0, self.dt, 0],
                            [0, 1, 0, self.dt],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])

        
        self.G = np.matrix([[(self.dt**2)/2, 0],
                            [0,(self.dt**2)/2],
                            [self.dt,0],
                            [0,self.dt]])

        
        self.H = np.matrix([[1, 0, 0, 0],
                            [0, 1, 0, 0]])


        self.Q = np.matrix([[(self.dt**4)/4, 0, (self.dt**3)/2, 0],
                            [0, (self.dt**4)/4, 0, (self.dt**3)/2],
                            [(self.dt**3)/2, 0, self.dt**2, 0],
                            [0, (self.dt**3)/2, 0, self.dt**2]]) * std_acc**2


        self.R = np.matrix([[x_std_meas**2,0],
                           [0, y_std_meas**2]])

        
        self.P = np.eye(self.F.shape[1])

    def predict(self):
     
        self.x = np.dot(self.F, self.x) + np.dot(self.G, self.u)
        
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        
        return self.x[0:2]

    def update(self, z):

        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  

        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))   

        I = np.eye(self.H.shape[1])

        self.P = (I - (K * self.H)) * self.P
        
        return self.x[0:2]

In [3]:
def detect(frame,debugMode):
    
    #Convert to grayscale image
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if (debugMode):
        cv2.imshow('gray', gray)

    #Edge detection of object
    img_edges = cv2.Canny(gray,  50, 190, 3)
    if (debugMode):
        cv2.imshow('img_edges', img_edges)

    # Convert to binary image
    ret, img_thresh = cv2.threshold(img_edges, 254, 255,cv2.THRESH_BINARY)
    if (debugMode):
        cv2.imshow('img_thresh', img_thresh)

    #Find contours
    contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    min_radius_thresh= 3
    max_radius_thresh= 30

    centers=[]
    for c in contours:
       
        (x, y), radius = cv2.minEnclosingCircle(c)
        radius = int(radius)

        if (radius > min_radius_thresh) and (radius < max_radius_thresh):
            centers.append(np.array([[x], [y]]))
    cv2.imshow('contours', img_thresh)
    return centers

In [5]:
def main():

    #Create opencv video capture object
    VideoCap = cv2.VideoCapture('OneBall.avi')
    #print(VideoCap)

   
    ControlSpeedVar = 100  
    HiSpeed = 100
    
    frame_width = int(VideoCap.get(3))
    frame_height = int(VideoCap.get(4))
   
    size = (frame_width, frame_height)
    
    result = cv2.VideoWriter('output_ball.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

###Attention
    KF = KalmanFilter(0.1, 1, 1, 1, 0.1,0.1)
###Attention

    debugMode=1

    while(True):
        # Read frame
        ret, frame = VideoCap.read()

        centers = detect(frame,debugMode)

        if (len(centers) > 0):
            
####Attention
            # Predict
            (x, y) = KF.predict()
            # Update
            #Input and Output coordinates are to be fed and fetched from here
            (x1, y1) = KF.update(centers[0]) #Input coordinates centers[0], Estimated output position coordinates (x1, y1)
####Attention

            #Mark the estimated position in the video
            cv2.rectangle(frame, (int(x1 - 15), int(y1 - 15)), (int(x1 + 15), int(y1 + 15)), (0, 0, 255), 2)

        result.write(frame)
        cv2.imshow('image', frame)
        

        if cv2.waitKey(2) & 0xFF == ord('q'):
            VideoCap.release()
            result.release()
            cv2.destroyAllWindows()
            break

        cv2.waitKey(HiSpeed-ControlSpeedVar+1)


if __name__ == "__main__":
    main()

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
